# Production-Grade RAG: From Vanilla to Autonomous Agents



## Part 1: Setup & Challenge



### 1.1 Environment Setup



In [ ]:
# !pip install -U langchain langgraph langchain_openai chromadb beautifulsoup4 rank_bm25 lxml sentence-transformers cross-encoder ragas arxiv rich sec-api unstructured[html] tavily-python

import os
import re
import json
from getpass import getpass
from pprint import pprint
import uuid
from typing import List, Dict, TypedDict, Literal, Optional

# Set API keys securely
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Enter your {var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")
_set_env("TAVILY_API_KEY")

# Configure LangSmith
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Advanced-Deep-Thinking-RAG-v2"

# Central config
config = {
    "data_dir": "./data",
    "vector_store_dir": "./vector_store",
    "llm_provider": "openai",
    "reasoning_llm": "gpt-4o",
    "fast_llm": "gpt-4o-mini",
    "embedding_model": "text-embedding-3-small",
    "reranker_model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "max_reasoning_iterations": 7,
    "top_k_retrieval": 10,
    "top_n_rerank": 3,
}

# Create directories
os.makedirs(config["data_dir"], exist_ok=True)
os.makedirs(config["vector_store_dir"], exist_ok=True)

print("✓ Environment setup complete")


### 1.2 Download NVIDIA 10-K Filing



In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.docstore.document import Document

def download_and_parse_10k(url, doc_path_raw, doc_path_clean):
    if os.path.exists(doc_path_clean):
        print(f"✓ Cleaned 10-K exists: {doc_path_clean}")
        return

    print(f"Downloading 10-K from {url}...")
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    with open(doc_path_raw, 'w', encoding='utf-8') as f:
        f.write(response.text)
    
    # Parse and clean HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    for table in soup.find_all('table'):
        table.decompose()

    text = ''
    for p in soup.find_all(['p', 'div', 'span']):
        text += p.get_text(strip=True) + '\n\n'
    
    clean_text = re.sub(r'\n{3,}', '\n\n', text).strip()
    clean_text = re.sub(r'\s{2,}', ' ', clean_text).strip()
    
    with open(doc_path_clean, 'w', encoding='utf-8') as f:
        f.write(clean_text)
    print(f"✓ Clean text saved: {doc_path_clean}")

# Download NVIDIA 2023 10-K
url_10k = "https://www.sec.gov/Archives/edgar/data/1045810/000104581023000017/nvda-20230129.htm"
doc_path_raw = os.path.join(config["data_dir"], "nvda_10k_2023_raw.html")
doc_path_clean = os.path.join(config["data_dir"], "nvda_10k_2023_clean.txt")

download_and_parse_10k(url_10k, doc_path_raw, doc_path_clean)

with open(doc_path_clean, 'r', encoding='utf-8') as f:
    print("Sample content:", f.read(1000)[:500] + "...")


### 1.3 Challenge Query



In [ ]:
complex_query = """
Based on NVIDIA's 2023 10-K filing, identify their key risks related to competition.
Then, find recent news (post-filing, from 2024) about AMD's AI chip strategy and 
explain how this new strategy directly addresses or exacerbates one of NVIDIA's stated risks.
"""
print("Challenge query set")


## Part 2: Baseline RAG Pipeline



### 2.1 Document Loading & Chunking



In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader(doc_path_clean, encoding='utf-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
doc_chunks = text_splitter.split_documents(documents)
print(f"✓ Split into {len(doc_chunks)} chunks")


### 2.2 Vector Store Creation



In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model=config['embedding_model'])
baseline_vector_store = Chroma.from_documents(
    documents=doc_chunks,
    embedding=embedding_function
)
baseline_retriever = baseline_vector_store.as_retriever(search_kwargs={"k": 3})
print(f"✓ Vector store created with {baseline_vector_store._collection.count()} embeddings")


### 2.3 Simple RAG Chain



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer based only on context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model=config["fast_llm"], temperature=0)

def format_docs(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

baseline_rag_chain = (
    {"context": baseline_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("✓ Baseline RAG chain ready")


### 2.4 Baseline Failure



In [ ]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()
print("Testing baseline RAG...")
baseline_result = baseline_rag_chain.invoke(complex_query)

console.print("--- BASELINE OUTPUT ---")
console.print(Markdown(baseline_result))


## Part 3: Advanced RAG Components



### 3.1 State Management



In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Step(BaseModel):
    sub_question: str
    justification: str
    tool: Literal["search_10k", "search_web"]
    keywords: List[str]
    document_section: Optional[str]

class Plan(BaseModel):
    steps: List[Step]

class PastStep(TypedDict):
    step_index: int
    sub_question: str
    retrieved_docs: List[Document]
    summary: str

class RAGState(TypedDict):
    original_question: str
    plan: Plan
    past_steps: List[PastStep]
    current_step_index: int
    retrieved_docs: List[Document]
    reranked_docs: List[Document]
    synthesized_context: str
    final_answer: str

print("✓ State management classes defined")


### 3.2 Tool-Aware Planner



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a research planner. Decompose queries into steps and assign tools:
    1. search_10k: For NVIDIA 10-K specific info
    2. search_web: For recent news/external info
    
    For search_10k steps, suggest relevant 10-K sections."""),
    ("human", "Query: {question}")
])

reasoning_llm = ChatOpenAI(model=config["reasoning_llm"], temperature=0)
planner_agent = planner_prompt | reasoning_llm.with_structured_output(Plan)

# Test planner
test_plan = planner_agent.invoke({"question": complex_query})
print("✓ Planner created")
print(f"Plan has {len(test_plan.steps)} steps")


### 3.3 Query Rewriter



In [ ]:
from langchain_core.output_parsers import StrOutputParser

query_rewriter_prompt = ChatPromptTemplate.from_messages([
    ("system", "Optimize search queries for retrieval systems."),
    ("human", """Sub-question: {sub_question}
Keywords: {keywords}
Past context: {past_context}
Rewritten query:""")
])

query_rewriter_agent = query_rewriter_prompt | reasoning_llm | StrOutputParser()
print("✓ Query rewriter ready")


### 3.4 Add Metadata for Filtered Search



In [ ]:
print("Adding section metadata to chunks...")
section_pattern = r"(ITEM\s+\d[A-Z]?\.\s*.*?)(?=\nITEM\s+\d[A-Z]?\.|$)"
raw_text = documents[0].page_content

section_titles = re.findall(section_pattern, raw_text, re.IGNORECASE | re.DOTALL)
section_titles = [title.strip().replace('\n', ' ') for title in section_titles]

sections_content = re.split(section_pattern, raw_text, flags=re.IGNORECASE | re.DOTALL)
sections_content = [c.strip() for c in sections_content if c.strip() and not c.strip().lower().startswith('item ')]

doc_chunks_with_metadata = []
for i, content in enumerate(sections_content):
    section_title = section_titles[i]
    section_chunks = text_splitter.split_text(content)
    for chunk in section_chunks:
        doc_chunks_with_metadata.append(
            Document(
                page_content=chunk,
                metadata={
                    "section": section_title,
                    "source_doc": doc_path_clean,
                    "id": str(uuid.uuid4())
                }
            )
        )

print(f"✓ Created {len(doc_chunks_with_metadata)} chunks with metadata")


### 3.5 Retrieval Supervisor



In [ ]:
class RetrievalDecision(BaseModel):
    strategy: Literal["vector_search", "keyword_search", "hybrid_search"]
    justification: str

retrieval_supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", """Choose best retrieval strategy:
    1. vector_search: Conceptual/semantic queries
    2. keyword_search: Specific terms/codes
    3. hybrid_search: Default, combines both"""),
    ("human", "Query: {sub_question}")
])

retrieval_supervisor_agent = retrieval_supervisor_prompt | reasoning_llm.with_structured_output(RetrievalDecision)
print("✓ Retrieval supervisor ready")


### 3.6 Advanced Vector Store



In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi

advanced_vector_store = Chroma.from_documents(
    documents=doc_chunks_with_metadata,
    embedding=embedding_function
)

# Build BM25 index
tokenized_corpus = [doc.page_content.split(" ") for doc in doc_chunks_with_metadata]
doc_ids = [doc.metadata["id"] for doc in doc_chunks_with_metadata]
doc_map = {doc.metadata["id"]: doc for doc in doc_chunks_with_metadata}
bm25 = BM25Okapi(tokenized_corpus)

def vector_search_only(query: str, section_filter: str = None, k: int = 10):
    filter_dict = {"section": section_filter} if section_filter and "Unknown" not in section_filter else None
    return advanced_vector_store.similarity_search(query, k=k, filter=filter_dict)

def bm25_search_only(query: str, k: int = 10):
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    top_k_indices = np.argsort(bm25_scores)[::-1][:k]
    return [doc_map[doc_ids[i]] for i in top_k_indices]

def hybrid_search(query: str, section_filter: str = None, k: int = 10):
    bm25_docs = bm25_search_only(query, k=k)
    semantic_docs = vector_search_only(query, section_filter=section_filter, k=k)
    
    # Reciprocal Rank Fusion
    all_docs = {doc.metadata["id"]: doc for doc in bm25_docs + semantic_docs}.values()
    ranked_lists = [
        [doc.metadata["id"] for doc in bm25_docs],
        [doc.metadata["id"] for doc in semantic_docs]
    ]
    
    rrf_scores = {}
    for doc_list in ranked_lists:
        for i, doc_id in enumerate(doc_list):
            if doc_id not in rrf_scores:
                rrf_scores[doc_id] = 0
            rrf_scores[doc_id] += 1 / (i + 61)
    
    sorted_doc_ids = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
    return [doc_map[doc_id] for doc_id in sorted_doc_ids[:k]]

print("✓ Advanced retrieval functions ready")


### 3.7 Reranker



In [ ]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder(config["reranker_model"])

def rerank_documents_function(query: str, documents: List[Document]) -> List[Document]:
    if not documents:
        return []
    pairs = [(query, doc.page_content) for doc in documents]
    scores = reranker.predict(pairs)
    
    doc_scores = list(zip(documents, scores))
    doc_scores.sort(key=lambda x: x[1], reverse=True)
    
    return [doc for doc, score in doc_scores[:config["top_n_rerank"]]]

print("✓ Reranker ready")


### 3.8 Context Distiller



In [ ]:
distiller_prompt = ChatPromptTemplate.from_messages([
    ("system", "Synthesize retrieved documents into concise context."),
    ("human", "Question: {question}\nDocuments:\n{context}")
])

distiller_agent = distiller_prompt | reasoning_llm | StrOutputParser()
print("✓ Distiller ready")


### 3.9 Web Search Tool



In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

def web_search_function(query: str) -> List[Document]:
    results = web_search_tool.invoke({"query": query})
    return [Document(page_content=res["content"], metadata={"source": res["url"]}) for res in results]

print("✓ Web search ready")


### 3.10 Reflection & Policy Agents



In [ ]:
# Reflection agent
reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", "Write one-sentence summary of findings."),
    ("human", "Sub-question: {sub_question}\nContext:\n{context}")
])
reflection_agent = reflection_prompt | reasoning_llm | StrOutputParser()

# Policy agent
class Decision(BaseModel):
    next_action: Literal["CONTINUE_PLAN", "FINISH"]
    justification: str

policy_prompt = ChatPromptTemplate.from_messages([
    ("system", """Decide next action based on research progress:
    - FINISH if question answered
    - CONTINUE_PLAN if more info needed"""),
    ("human", "Question: {question}\nPlan:\n{plan}\nHistory:\n{history}")
])
policy_agent = policy_prompt | reasoning_llm.with_structured_output(Decision)

print("✓ Reflection & policy agents ready")


## Part 4: LangGraph Orchestration



### 4.1 Graph Nodes



In [ ]:
from langgraph.graph import StateGraph, END

def get_past_context_str(past_steps: List[PastStep]) -> str:
    return "\n\n".join([f"Step {s['step_index']}: {s['sub_question']}\nSummary: {s['summary']}" for s in past_steps])

# Define nodes
def plan_node(state: RAGState) -> Dict:
    print("🧠 Generating plan...")
    plan = planner_agent.invoke({"question": state["original_question"]})
    return {"plan": plan, "current_step_index": 0, "past_steps": []}

def retrieval_node(state: RAGState) -> Dict:
    current_step = state["plan"].steps[state["current_step_index"]]
    print(f"🔍 Retrieving (Step {state['current_step_index'] + 1})...")
    
    past_context = get_past_context_str(state['past_steps'])
    rewritten_query = query_rewriter_agent.invoke({
        "sub_question": current_step.sub_question,
        "keywords": current_step.keywords,
        "past_context": past_context
    })
    
    retrieval_decision = retrieval_supervisor_agent.invoke({"sub_question": rewritten_query})
    
    if retrieval_decision.strategy == 'vector_search':
        docs = vector_search_only(rewritten_query, current_step.document_section, config['top_k_retrieval'])
    elif retrieval_decision.strategy == 'keyword_search':
        docs = bm25_search_only(rewritten_query, config['top_k_retrieval'])
    else:
        docs = hybrid_search(rewritten_query, current_step.document_section, config['top_k_retrieval'])
    
    return {"retrieved_docs": docs}

def web_search_node(state: RAGState) -> Dict:
    current_step = state["plan"].steps[state["current_step_index"]]
    print(f"🌐 Web search (Step {state['current_step_index'] + 1})...")
    
    past_context = get_past_context_str(state['past_steps'])
    rewritten_query = query_rewriter_agent.invoke({
        "sub_question": current_step.sub_question,
        "keywords": current_step.keywords,
        "past_context": past_context
    })
    
    return {"retrieved_docs": web_search_function(rewritten_query)}

def rerank_node(state: RAGState) -> Dict:
    print("🎯 Reranking...")
    current_step = state["plan"].steps[state["current_step_index"]]
    return {"reranked_docs": rerank_documents_function(current_step.sub_question, state["retrieved_docs"])}

def compression_node(state: RAGState) -> Dict:
    print("✂️ Distilling context...")
    current_step = state["plan"].steps[state["current_step_index"]]
    context = format_docs(state["reranked_docs"])
    synthesized = distiller_agent.invoke({"question": current_step.sub_question, "context": context})
    return {"synthesized_context": synthesized}

def reflection_node(state: RAGState) -> Dict:
    print("🤔 Reflecting...")
    current_step = state["plan"].steps[state["current_step_index"]]
    summary = reflection_agent.invoke({
        "sub_question": current_step.sub_question,
        "context": state['synthesized_context']
    })
    
    new_step = {
        "step_index": state["current_step_index"] + 1,
        "sub_question": current_step.sub_question,
        "retrieved_docs": state['reranked_docs'],
        "summary": summary
    }
    
    return {
        "past_steps": state["past_steps"] + [new_step],
        "current_step_index": state["current_step_index"] + 1
    }

def final_answer_node(state: RAGState) -> Dict:
    print("✅ Generating final answer...")
    
    final_context = ""
    for i, step in enumerate(state['past_steps']):
        final_context += f"\n--- Step {i+1} ---\n"
        for doc in step['retrieved_docs']:
            source = doc.metadata.get('section') or doc.metadata.get('source')
            final_context += f"Source: {source}\nContent: {doc.page_content}\n\n"
    
    final_answer_prompt = ChatPromptTemplate.from_messages([
        ("system", "Synthesize research into answer with citations."),
        ("human", "Question: {question}\nContext:\n{context}")
    ])
    
    final_answer_agent = final_answer_prompt | reasoning_llm | StrOutputParser()
    answer = final_answer_agent.invoke({
        "question": state['original_question'],
        "context": final_context
    })
    
    return {"final_answer": answer}

print("✓ Graph nodes defined")


### 4.2 Conditional Edges



In [ ]:
def route_by_tool(state: RAGState) -> str:
    current_step = state["plan"].steps[state["current_step_index"]]
    return current_step.tool

def should_continue_node(state: RAGState) -> str:
    print("🚦 Evaluating policy...")
    
    if state["current_step_index"] >= len(state["plan"].steps):
        print("  → Plan complete")
        return "finish"
    
    if state["current_step_index"] >= config["max_reasoning_iterations"]:
        print("  → Max iterations")
        return "finish"
    
    if not state["reranked_docs"]:
        print("  → No documents found")
        return "continue"
    
    history = get_past_context_str(state['past_steps'])
    plan_str = json.dumps([s.dict() for s in state['plan'].steps])
    decision = policy_agent.invoke({
        "question": state["original_question"],
        "plan": plan_str,
        "history": history
    })
    
    print(f"  → Decision: {decision.next_action}")
    return "finish" if decision.next_action == "FINISH" else "continue"

print("✓ Edge logic defined")


### 4.3 Build Graph



In [ ]:
graph = StateGraph(RAGState)

# Add nodes
graph.add_node("plan", plan_node)
graph.add_node("retrieve_10k", retrieval_node)
graph.add_node("retrieve_web", web_search_node)
graph.add_node("rerank", rerank_node)
graph.add_node("compress", compression_node)
graph.add_node("reflect", reflection_node)
graph.add_node("generate_final_answer", final_answer_node)

# Define edges
graph.set_entry_point("plan")
graph.add_conditional_edges(
    "plan",
    route_by_tool,
    {"search_10k": "retrieve_10k", "search_web": "retrieve_web"},
)
graph.add_edge("retrieve_10k", "rerank")
graph.add_edge("retrieve_web", "rerank")
graph.add_edge("rerank", "compress")
graph.add_edge("compress", "reflect")
graph.add_conditional_edges(
    "reflect",
    should_continue_node,
    {"continue": "plan", "finish": "generate_final_answer"},
)
graph.add_edge("generate_final_answer", END)

deep_thinking_rag_graph = graph.compile()
print("✓ Graph compiled")


### 4.4 Visualize



In [ ]:
try:
    from IPython.display import Image, display
    png_image = deep_thinking_rag_graph.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}")


## Part 5: Run Advanced Pipeline



### 5.1 Execute



In [ ]:
print("🚀 Starting advanced RAG...")
graph_input = {"original_question": complex_query}
final_state = None

for chunk in deep_thinking_rag_graph.stream(graph_input, stream_mode="values"):
    final_state = chunk
    print(f"Step completed")

print("\n✅ Graph execution complete")


### 5.2 Show Results



In [ ]:
console.print("--- ADVANCED RAG OUTPUT ---")
console.print(Markdown(final_state['final_answer']))


## Part 6: Evaluation



### 6.1 Setup Evaluation



In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall, faithfulness, answer_correctness
import pandas as pd

ground_truth = """
NVIDIA's 2023 10-K lists intense competition and rapid technological change as key risks.
This risk is exacerbated by AMD's 2024 strategy, specifically the MI300X AI accelerator,
which directly competes with NVIDIA's H100 and has been adopted by major cloud providers.
"""

# Get contexts for evaluation
retrieved_docs_baseline = baseline_retriever.invoke(complex_query)
baseline_contexts = [[doc.page_content for doc in retrieved_docs_baseline]]

advanced_contexts_flat = []
for step in final_state['past_steps']:
    advanced_contexts_flat.extend([doc.page_content for doc in step['retrieved_docs']])
advanced_contexts = [list(set(advanced_contexts_flat))]

eval_data = {
    'question': [complex_query, complex_query],
    'answer': [baseline_result, final_state['final_answer']],
    'contexts': baseline_contexts + advanced_contexts,
    'ground_truth': [ground_truth, ground_truth]
}

eval_dataset = Dataset.from_dict(eval_data)
print("✓ Evaluation dataset ready")


### 6.2 Run Evaluation



In [ ]:
metrics = [context_precision, context_recall, faithfulness, answer_correctness]
result = evaluate(eval_dataset, metrics=metrics, is_async=False)

results_df = result.to_pandas()
results_df.index = ['baseline', 'advanced']
print("\n📊 Evaluation Results:")
print(results_df[['context_precision', 'context_recall', 'faithfulness', 'answer_correctness']])


## Part 7: Production Optimizations



### 7.1 Caching



In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# Enable caching for production
set_llm_cache(InMemoryCache())
print("✓ LLM caching enabled")


### 7.2 Provenance Tracking



In [ ]:
# Already implemented in final_answer_node
print("✓ Citations and provenance built-in")
